Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [17]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2

In [18]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 20

### Model Architecture

In [37]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,
                             
                tf.keras.layers.Conv2D(32, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Flatten(),                                   # Flatten to vector
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(52, activation='softmax')
                ]
        
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Dataset Loading

In [20]:
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)
    
new_data = {}
for key, inner_dict in data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['class_label']
    new_data[img_path] = value

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((new_data.keys(), new_data.values()))

def load_train_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [8]:
# this cell it optional
# this checks that the dataset is balanced
import collections
v_counts = collections.Counter()
t_counts = collections.Counter()

for _, label in val_dataset:
    label = label.numpy().tolist() 
    v_counts.update(label)

for _, label in train_dataset:
    label = label.numpy().tolist() 
    t_counts.update(label)
    
print("Label counts in validation dataset:")
for i in range(52):
    print(f"Label {i}: {v_counts[i]}, {t_counts[i]}")

2025-04-29 21:21:48.904347: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/5.png; No such file or directory
2025-04-29 21:21:48.904360: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/0.png; No such file or directory
2025-04-29 21:21:48.904369: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/2.png; No such file or directory
2025-04-29 21:21:48.904378: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/1.png; No such file or directory
2025-04-29 21:21:48.904442: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: imgs/single/9.png; No such file or directory
2025-04-29 21:21:48.904456: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES fai

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::Prefetch::FiniteSkip::Prefetch::BatchV2::Shuffle::ParallelMapV2: imgs/single/0.png; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

### Model Loading and Training

In [39]:
model = CardPredictor()
model.build((None, 300, 300, 3))
model.load_weights("model_weights_e80_a88.weights.h5")

model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])

In [40]:
model.fit(train_dataset, 
          validation_data=val_dataset, 
          epochs=NUM_EPOCHS, 
          verbose=1)

Epoch 1/20
 11/175 ━━━━━━━━━━━━━━━━━━━━ 35s 217ms/step - accuracy: 0.8960 - loss: 27802.2266

KeyboardInterrupt: 

In [24]:
model.save_weights('new_weights.weights.h5')

In [31]:
import h5py

with h5py.File('model_weights_e40_a82.weights.h5', 'r') as f:
    def recursively_print(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"{name}: {obj.shape}")
    
    f.visititems(recursively_print)
    
    
print("New weights:")
print("__________________________________:")

with h5py.File('new_weights.weights.h5', 'r') as f:
    def recursively_print(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"{name}: {obj.shape}")
    
    f.visititems(recursively_print)

architecture/batch_normalization/vars/0: (32,)
architecture/batch_normalization/vars/1: (32,)
architecture/batch_normalization/vars/2: (32,)
architecture/batch_normalization/vars/3: (32,)
architecture/batch_normalization_1/vars/0: (64,)
architecture/batch_normalization_1/vars/1: (64,)
architecture/batch_normalization_1/vars/2: (64,)
architecture/batch_normalization_1/vars/3: (64,)
architecture/batch_normalization_2/vars/0: (128,)
architecture/batch_normalization_2/vars/1: (128,)
architecture/batch_normalization_2/vars/2: (128,)
architecture/batch_normalization_2/vars/3: (128,)
architecture/batch_normalization_3/vars/0: (256,)
architecture/batch_normalization_3/vars/1: (256,)
architecture/batch_normalization_3/vars/2: (256,)
architecture/batch_normalization_3/vars/3: (256,)
architecture/batch_normalization_4/vars/0: (512,)
architecture/batch_normalization_4/vars/1: (512,)
architecture/batch_normalization_4/vars/2: (512,)
architecture/batch_normalization_4/vars/3: (512,)
architecture/con

## Video capture and classification

In [25]:
model = CardPredictor()
model.build((None, 300, 300, 3))
model.load_weights('model_weights.weights.h5')

/Users/RioVK/miniforge3/envs/cs1430/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
# Load model
model = CardPredictor()
model.build((None, 300, 300, 3))
model.load_weights('model_weights.weights.h5')

values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
suits = ['spade', 'heart', 'diamond', 'club']
class_names = [f"{v} of {s}" for v in values for s in suits]
# === Start camera ===
cam = cv2.VideoCapture(0)

if not cam.isOpened():
    print("Error: Could not open camera.")
    exit()

print("Press 'c' to capture and classify a card.")
print("Press 'q' to quit.")

while True:
    ret, frame = cam.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    # Show the camera feed
    cv2.imshow("Live Feed - Press 'c' to classify, 'q' to quit", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        print("Exiting.")
        break
    elif key == ord('c'):
        print("Capturing frame...")

        # Preprocess the frame
        img = cv2.resize(frame, (300, 300))
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)

        # Predict
        pred = model(img, training=False)
        predicted_class = tf.argmax(pred, axis=1).numpy()[0]
        # check dataset to fix the indexing here.
        class_name = class_names[predicted_class]

        # Overlay prediction and display result on the top right corner of the image. 
        display_frame = frame.copy()
        cv2.putText(display_frame, f"Prediction: {class_name}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("Prediction", display_frame)
        cv2.waitKey(1500)  # Show prediction for 1.5 seconds

cam.release()
cv2.destroyAllWindows()

2025-04-29 21:34:38.881 python[10625:18927309] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Press 'c' to capture and classify a card.
Press 'q' to quit.


2025-04-29 21:34:42.497 python[10625:18927309] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-29 21:34:42.497 python[10625:18927309] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Capturing frame...
Exiting.


: 